In [6]:
import cv2
import numpy as np

# Open webcam
cap = cv2.VideoCapture(0)

# Create a blank white image as canvas
canvas = np.ones((480, 640, 3), np.uint8) * 255

# HSV range for black marker (fixed values)
lower = np.array([0, 50, 172])     # LH = 0, LS = 50, LV = 172
upper = np.array([179, 255, 255])  # UH = 179, US = 255, UV = 255

# Previous points
prev_x, prev_y = 0, 0

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    frame = cv2.flip(frame, 1)  # Flip for mirror effect
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Create mask for black marker
    mask = cv2.inRange(hsv, lower, upper)

    # Find contours
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if contours:
        # Pick the largest contour
        c = max(contours, key=cv2.contourArea)
        (x, y, w, h) = cv2.boundingRect(c)

        # Get center of the contour
        cx, cy = x + w // 2, y + h // 2

        # Draw a circle on the marker position
        cv2.circle(frame, (cx, cy), 5, (0, 0, 255), -1)

        # Draw on the canvas
        if prev_x == 0 and prev_y == 0:
            prev_x, prev_y = cx, cy
        cv2.line(canvas, (prev_x, prev_y), (cx, cy), (0, 0, 255), 5)

        prev_x, prev_y = cx, cy
    else:
        prev_x, prev_y = 0, 0  # reset if marker not found

    # Merge canvas with webcam
    combined = cv2.addWeighted(frame, 0.7, canvas, 0.3, 0)

    cv2.imshow("Virtual Paint Brush", combined)
    cv2.imshow("Mask", mask)

    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):  # Quit
        break
    elif key == ord('c'):  # Clear canvas
        canvas = np.ones((480, 640, 3), np.uint8) * 255

cap.release()
cv2.destroyAllWindows()
